In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [2]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [3]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [4]:
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
returns = merge(returns, log1p.(data_source[:T_Bill])./4)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
term = data_source[:Yeld_10Y] ./4 .- data_source[:T_Bill] ./4
dy = data_source[:Dividend_Yeld] #log.(data_source[:Dividend_Yeld])
data = merge(real_returns, dy, term, cpi)

291×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1952-03-01 │      1.98335e-5 │     0.0107337 │  0.00772445 │     0.0602359 │ ⋯
│ 1952-06-01 │       0.0581411 │    0.00263202 │ -0.00708528 │      0.057815 │ ⋯
│ 1952-09-01 │      -0.0185027 │    -0.0173351 │  0.00423886 │     0.0592197 │ ⋯
│ 1952-12-01 │       0.0932838 │     0.0220972 │   0.0089235 │     0.0538579 │ ⋯
│ 1953-03-01 │      -0.0437093 │   -0.00972202 │  0.00497517 │     0.0571967 │ ⋯
│ 1953-06-01 │      -0.0100139 │    -0.0219485 │ -0.00227055 │     0.0584603 │ ⋯
│ 1953-09-01 │     -0.00574595 │     0.0298874 │ -0.00299956 │     0.0596579 │ ⋯
│ 1953-12-01 │        0.078226 │     0.0248173 │  0.00767892 │     0.0572141 │ ⋯
│     ⋮      │        ⋮        │ 

In [5]:
CSV.write("usa_data_transformed.csv", data)

"usa_data_transformed.csv"

In [18]:
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [19]:
returns_summarystats(to(from(returns, start),end_d), 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.144099,0.782737,0.0963943,0.261158,-1.01471,-0.845921,-0.0423317,0.0953652,1.77521,0.668946,0.142769
Price_10Y,0.0832445,0.569087,0.044665,0.156403,-0.289949,0.371142,-0.0740101,0.0442422,0.221941,0.536551,0.0280389
T_Bill,0.0101058,0.0582689,0.0227848,0.045069,9.9995e-5,0.292382,0.00149888,0.968883,-1.50635,2.25464,0.0172994


In [20]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.0717569,0.193813,0.0178775,0.0610642,-0.238141,-0.699168,-0.0145302,0.0849045,1.2878,0.24914,0.0283261
Price_10Y_CPI,0.0448954,0.167664,0.0049452,0.0347394,-0.0960374,0.432308,-0.0258376,0.0666902,0.789709,0.110149,0.00203425
T_Bill_CPI,0.00893438,0.025467,-0.000524846,0.00528326,-0.026825,-0.318287,-0.00503521,0.302165,0.492507,-0.0587446,0.000505661
Dividend_Yeld,0.00435984,0.0323637,0.0189393,0.0207695,0.0112582,0.765491,0.0164025,0.918403,0.717749,4.34404,0.0185857
Yeld_10Y_T_Bill,0.00293746,0.0095,0.00388508,0.006225,-0.0034,-0.236784,0.0017125,0.874579,-0.53084,1.3226,0.00395
CPI,0.00774615,0.0279226,0.00622104,0.00985738,-0.025392,-0.34528,0.00268687,0.0727647,2.47992,0.803114,0.00575376


In [21]:
returns_summarystats(s_data, 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.143514,0.775253,0.0715101,0.244257,-0.952565,-0.699168,-0.0581207,0.0849045,1.2878,0.498281,0.113304
Price_10Y_CPI,0.0897907,0.670655,0.0197808,0.138958,-0.38415,0.432308,-0.10335,0.0666902,0.789709,0.220299,0.00813698
T_Bill_CPI,0.0178688,0.101868,-0.00209938,0.021133,-0.1073,-0.318287,-0.0201408,0.302165,0.492507,-0.117489,0.00202265
Dividend_Yeld,0.00871968,0.129455,0.0757573,0.0830779,0.0450328,0.765491,0.06561,0.918403,0.717749,8.68808,0.0743426
Yeld_10Y_T_Bill,0.00587492,0.038,0.0155403,0.0249,-0.0136,-0.236784,0.00685,0.874579,-0.53084,2.6452,0.0158
CPI,0.0154923,0.11169,0.0248842,0.0394295,-0.101568,-0.34528,0.0107475,0.0727647,2.47992,1.60623,0.0230151


In [22]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI,std
Total_Stock_CPI,-0.0295,0.1152,0.0788,-0.8405,4.0279,-5.652,-1.5289,0.07
Price_10Y_CPI,-0.0077,-0.1285,-0.0358,2.8038,-0.488,3.675,1.779,0.0427
T_Bill_CPI,-0.0122,-0.0249,-0.0146,1.0719,0.3155,0.119,1.0222,0.0073
Dividend_Yeld,0.0012,-0.003,-0.0016,0.002,0.9069,0.0919,0.0168,0.0015
Yeld_10Y_T_Bill,0.0003,0.0017,0.0066,-0.0515,0.0091,0.887,-0.0329,0.0012
CPI,0.0118,0.0262,0.0089,-0.0735,-0.3003,-0.0871,-0.0139,0.0075


,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI
Total_Stock_CPI,1.0,-0.16,0.03,-0.91,0.02,-0.01
Price_10Y_CPI,-0.16,1.0,0.44,0.23,-0.58,-0.49
T_Bill_CPI,0.03,0.44,1.0,0.16,-0.14,-0.99
Dividend_Yeld,-0.91,0.23,0.16,1.0,-0.07,-0.17
Yeld_10Y_T_Bill,0.02,-0.58,-0.14,-0.07,1.0,0.07
CPI,-0.01,-0.49,-0.99,-0.17,0.07,1.0


In [23]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

"Total_Stock_CPI coefficients"

Quantiles
       parameters       2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol    Float64   Float64   Float64   Float64   Float64 

            const    -0.0974   -0.0528   -0.0285   -0.0053    0.0401
  Total_Stock_CPI    -0.0688    0.0542    0.1154    0.1798    0.3000
    Price_10Y_CPI    -0.2592   -0.0379    0.0763    0.1959    0.4230
       T_Bill_CPI    -3.8871   -1.9094   -0.8506    0.1858    2.2261
    Dividend_Yeld     0.6558    2.8688    3.9912    5.1348    7.3459
  Yeld_10Y_T_Bill   -11.3680   -7.6475   -5.6374   -3.6805    0.1455
              CPI    -5.0041   -2.7558   -1.5636   -0.3478    1.9951


"Price_10Y_CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0500   -0.0219   -0.0077    0.0065    0.0346
  Total_Stock_CPI   -0.2414   -0.1659   -0.1280   -0.0894   -0.0155
    Price_10Y_CPI   -0.2471   -0.1093   -0.0368    0.0367    0.1774
       T_Bill_CPI    0.9082    2.1717    2.7992    3.4280    4.6716
    Dividend_Yeld   -2.5372   -1.1848   -0.4757    0.2187    1.5621
  Yeld_10Y_T_Bill    0.0163    2.4430    3.6570    4.8700    7.1259
              CPI   -0.3113    1.0357    1.7513    2.5126    3.9664


"T_Bill_CPI coefficients"

...

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0009   -0.0001    0.0003    0.0007    0.0015
  Total_Stock_CPI   -0.0016    0.0005    0.0017    0.0028    0.0050
    Price_10Y_CPI    0.0004    0.0045    0.0065    0.0086    0.0127
       T_Bill_CPI   -0.1050   -0.0695   -0.0514   -0.0335    0.0022
    Dividend_Yeld   -0.0503   -0.0112    0.0086    0.0290    0.0665
  Yeld_10Y_T_Bill    0.7879    0.8529    0.8873    0.9224    0.9893
              CPI   -0.0946   -0.0539   -0.0326   -0.0123    0.0282


"CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const    0.0045    0.0093    0.0118    0.0143    0.0191
  Total_Stock_CPI    0.0063    0.0194    0.0261    0.0329    0.0459
    Price_10Y_CPI   -0.0290   -0.0038    0.0086    0.0219    0.0455
       T_Bill_CPI   -0.3920   -0.1859   -0.0753    0.0346    0.2516
    Dividend_Yeld   -0.6540   -0.4234   -0.3042   -0.1781    0.0629
  Yeld_10Y_T_Bill   -0.7099   -0.3036   -0.0856    0.1257    0.5384
              CPI   -0.3943   -0.1455   -0.0150    0.1123    0.3573


"correlation matrix"

Quantiles
                       parameters      2.5%     25.0%     50.0%     75.0%      ⋯
                           Symbol   Float64   Float64   Float64   Float64   Fl ⋯

  Total_Stock_CPI_Total_Stock_CPI    0.0634    0.0687    0.0717    0.0751    0 ⋯
    Total_Stock_CPI_Price_10Y_CPI   -0.3311   -0.2172   -0.1549   -0.0930    0 ⋯
       Total_Stock_CPI_T_Bill_CPI   -0.1557   -0.0341    0.0288    0.0933    0 ⋯
    Total_Stock_CPI_Dividend_Yeld   -0.9386   -0.9224   -0.9125   -0.9011   -0 ⋯
  Total_Stock_CPI_Yeld_10Y_T_Bill   -0.1655   -0.0439    0.0199    0.0833    0 ⋯
              Total_Stock_CPI_CPI   -0.1906   -0.0712   -0.0071    0.0571    0 ⋯
    Price_10Y_CPI_Total_Stock_CPI   -0.3311   -0.2172   -0.1549   -0.0930    0 ⋯
      Price_10Y_CPI_Price_10Y_CPI    0.0387    0.0419    0.0438    0.0459    0 ⋯
         Price_10Y_CPI_T_Bill_CPI    0.2728    0.3842    0.4380    0.4873    0 ⋯
      Price_10Y_CPI_Dividend_Yeld    0.0459    0.1655    0.2279    0.2866    0 ⋯
    Price_10Y_CPI

In [24]:
vmodel.Y[end,:]

6-element Vector{Float64}:
  0.12159459085777868
  0.06515539231011935
  0.010346584402051042
  0.014635185560818083
 -0.0034000000000000002
  0.002421733143949112

In [25]:
names = colnames(s_data)
scenarios = simulate(vmodel,60)

for n in 1:6
    s_ann = annualise(scenarios[n,:,2:end],4)
    print_percentiles(s_ann,[0.05, 0.25, 0.5, 0.75, 0.95], string(names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0885403,-0.163752,-0.180772,-0.188086,-0.194003,-0.201019,-0.195448,-0.198498,-0.194239,-0.194368,-0.193536,-0.195998,-0.189286,-0.18742,-0.191244
0.25,0.0367743,-0.0248398,-0.0369324,-0.0436765,-0.0481122,-0.0518706,-0.0497868,-0.052605,-0.0489,-0.0497481,-0.0463912,-0.0485337,-0.0418272,-0.0455421,-0.0462951
0.5,0.129258,0.0765528,0.062854,0.0545621,0.0514334,0.0475172,0.0491178,0.0530701,0.0500241,0.0544128,0.0522046,0.0507457,0.0571615,0.0551806,0.055627
0.75,0.219951,0.173344,0.164214,0.158206,0.154935,0.149134,0.153594,0.15307,0.15467,0.154266,0.153782,0.150336,0.158611,0.155731,0.159499
0.95,0.350704,0.317774,0.306307,0.308505,0.300769,0.298501,0.300432,0.300972,0.301861,0.300827,0.299504,0.298739,0.309237,0.297456,0.307798


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.151035,-0.13261,-0.134053,-0.134542,-0.136575,-0.135592,-0.138177,-0.140837,-0.137808,-0.140243,-0.142881,-0.139865,-0.144991,-0.140575,-0.143596
0.25,-0.0749857,-0.0506027,-0.0507734,-0.0479678,-0.0469599,-0.0473484,-0.0482173,-0.0525916,-0.0515181,-0.0523519,-0.0544653,-0.0545381,-0.0547377,-0.053493,-0.0560736
0.5,-0.0195716,0.00481595,0.00999102,0.0117382,0.0124228,0.0129838,0.0120436,0.0097759,0.0110405,0.0103846,0.00895148,0.00552474,0.00647498,0.00859409,0.00768933
0.75,0.0344353,0.0649527,0.0694541,0.0728958,0.0727947,0.0728751,0.0733149,0.0708555,0.0723071,0.0719772,0.0687794,0.0676918,0.0668106,0.0688351,0.067711
0.95,0.111115,0.147833,0.158127,0.158986,0.161383,0.162851,0.16502,0.16005,0.161975,0.157321,0.156755,0.155745,0.157607,0.15735,0.153284


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0124519,-0.0151958,-0.0209669,-0.0252479,-0.0289037,-0.0315366,-0.0331029,-0.0362017,-0.037206,-0.0382489,-0.0397461,-0.0405987,-0.0409028,-0.0411034,-0.0412097
0.25,0.00227435,0.00133502,-0.00241783,-0.00542048,-0.00772349,-0.0102276,-0.0114989,-0.0137279,-0.0143021,-0.015908,-0.0171558,-0.0170424,-0.0176434,-0.0183367,-0.0187253
0.5,0.0123095,0.0134146,0.0103804,0.00853962,0.00650268,0.00522435,0.00367135,0.00208205,0.00116962,0.000491609,-0.000981181,-0.0015627,-0.00158472,-0.00184425,-0.00286358
0.75,0.02234,0.0247707,0.0231949,0.0219347,0.0209845,0.0200849,0.0185942,0.0175824,0.0171043,0.0159205,0.0150618,0.0147017,0.0138659,0.0140875,0.0133067
0.95,0.0367457,0.0417366,0.0422582,0.04223,0.0415983,0.0407174,0.0408235,0.0400082,0.0392391,0.0391735,0.0384834,0.0382881,0.0375955,0.0364429,0.0370051


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.0418802,0.0343175,0.0309907,0.0308737,0.030868,0.0311707,0.0320042,0.0327895,0.0337933,0.0341939,0.0347333,0.0356207,0.0359782,0.0365525,0.0373313
0.25,0.0491415,0.0451767,0.044117,0.0442071,0.044889,0.0462038,0.0474038,0.04838,0.0490611,0.050093,0.0510608,0.0521027,0.0525781,0.0530166,0.0533295
0.5,0.0541969,0.0529003,0.0532697,0.0539797,0.055082,0.0564924,0.0579959,0.0591792,0.0601814,0.0611827,0.0621527,0.0629945,0.0635334,0.0640444,0.0647174
0.75,0.059365,0.0608624,0.0625368,0.0637881,0.0653487,0.0670377,0.068529,0.0698523,0.0711724,0.072339,0.0731791,0.07405,0.0748912,0.0752815,0.0756877
0.95,0.0666165,0.072,0.0749538,0.0780152,0.0800715,0.0817376,0.0841799,0.0853068,0.0868538,0.0883428,0.0891328,0.089717,0.0910088,0.0914198,0.0915924


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0178174,-0.0193803,-0.0189516,-0.0181818,-0.0171232,-0.0167469,-0.0161328,-0.0157094,-0.0150105,-0.0142042,-0.0137706,-0.0132594,-0.0132662,-0.0131075,-0.0121504
0.25,-0.0123895,-0.0109958,-0.00918953,-0.00757865,-0.00635478,-0.00523672,-0.00446478,-0.00363275,-0.00275238,-0.00227486,-0.001569,-0.00103571,-0.000528262,-7.47159e-6,0.000373801
0.5,-0.0087212,-0.00521632,-0.00242456,-0.000454151,0.00129889,0.00280903,0.00396042,0.00481461,0.00566439,0.0063558,0.00706186,0.00773015,0.00827099,0.0086823,0.00894882
0.75,-0.00494639,0.000496347,0.00398354,0.00669434,0.00899212,0.0106119,0.0120348,0.0133325,0.0142156,0.0151205,0.0157378,0.0163197,0.0169744,0.0172568,0.0177626
0.95,0.000653558,0.00875804,0.0133871,0.017211,0.0198611,0.0218207,0.0234901,0.0251135,0.0264549,0.0272787,0.0284124,0.029292,0.0296205,0.0297879,0.0305134


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.00980772,0.00550615,0.00522247,0.00460777,0.00395123,0.00286054,0.00281142,0.0030216,0.0019865,0.000989938,0.00208655,0.00156822,0.00098231,0.00104802,0.000824781
0.25,0.0253564,0.0215256,0.0210244,0.0205581,0.0199808,0.0195207,0.0190948,0.0190179,0.0181317,0.0176761,0.018159,0.0177591,0.0173265,0.0173387,0.0171505
0.5,0.036056,0.0325572,0.0324219,0.0315989,0.0312496,0.030478,0.0301629,0.030279,0.0294851,0.0293266,0.0291115,0.0287918,0.0285725,0.0283442,0.0283911
0.75,0.0466883,0.0432917,0.043652,0.0429085,0.0421693,0.0417593,0.0411252,0.0416315,0.0405295,0.0403924,0.0407122,0.0402528,0.0398652,0.0394438,0.039634
0.95,0.0628809,0.0592977,0.0591822,0.0596068,0.0586587,0.0585049,0.0568234,0.0572313,0.0570798,0.057018,0.0565492,0.0561403,0.0569794,0.0553321,0.0555004


In [26]:
scenarios_b = simulate(bmodel,60, 10)
for n in 1:6
    s_ann = annualise(scenarios_b[n,:,:],4)
    print_percentiles(s_ann,[0.02, 0.25, 0.5, 0.75, 0.98], string(names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0174204,-0.48212,-0.585823,-0.626813,-0.659085,-0.680282,-0.696018,-0.706561,-0.721592,-0.733683,-0.743845,-0.758258,-0.766531,-0.781073,-0.793949
0.25,0.163383,-0.0738129,-0.110062,-0.126662,-0.135894,-0.140336,-0.14376,-0.147686,-0.148483,-0.14958,-0.150563,-0.15189,-0.150052,-0.151231,-0.152074
0.5,0.225936,0.0812407,0.0685918,0.0623576,0.0607452,0.0595392,0.060292,0.0610957,0.0611889,0.0612523,0.0640455,0.0623882,0.0645938,0.0653823,0.0672419
0.75,0.294302,0.242012,0.249202,0.25529,0.257365,0.259314,0.265544,0.270803,0.273176,0.274515,0.279857,0.280789,0.283847,0.285453,0.288516
0.98,0.47962,0.688825,0.755634,0.779013,0.801365,0.822509,0.839407,0.861319,0.884132,0.90472,0.922367,0.949205,0.984013,1.02096,1.04692


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.120865,-0.470829,-0.559616,-0.595154,-0.623737,-0.635942,-0.653074,-0.662131,-0.674018,-0.683711,-0.693685,-0.697538,-0.707806,-0.71449,-0.725642
0.25,0.00126584,-0.124933,-0.150402,-0.164422,-0.17553,-0.183599,-0.190954,-0.194,-0.198524,-0.200071,-0.20433,-0.205962,-0.210537,-0.213329,-0.213955
0.5,0.0484666,0.00723653,0.0110887,0.01119,0.0113292,0.0094759,0.00671627,0.00544745,0.00392564,0.00524662,0.00445977,0.00305528,0.00218749,0.00179957,0.00181504
0.75,0.0927445,0.136529,0.171339,0.186371,0.195481,0.200392,0.203215,0.204082,0.208126,0.20936,0.21125,0.211029,0.211463,0.213903,0.21498
0.98,0.196823,0.468608,0.570886,0.615104,0.646602,0.660121,0.666935,0.676034,0.684883,0.689062,0.694875,0.699336,0.700631,0.702936,0.706902


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0341149,-0.127093,-0.17512,-0.209937,-0.237904,-0.263151,-0.285055,-0.302464,-0.32247,-0.338587,-0.353996,-0.371078,-0.385951,-0.406217,-0.425574
0.25,0.00254036,-0.0293174,-0.0454985,-0.0572109,-0.065739,-0.0728482,-0.0783555,-0.083098,-0.0869971,-0.0897856,-0.0914979,-0.0934088,-0.0949258,-0.097359,-0.0985668
0.5,0.0188996,0.0138185,0.0112765,0.00892066,0.00718738,0.00564106,0.00453798,0.00333282,0.00229769,0.00195461,0.00209087,0.00195549,0.00154455,0.0015197,0.00145049
0.75,0.0354722,0.0570873,0.0680692,0.0747539,0.0801094,0.0838134,0.087,0.0895166,0.0924276,0.0948971,0.0964712,0.0972694,0.0990358,0.100997,0.102296
0.98,0.0715436,0.153334,0.196214,0.226506,0.253758,0.274914,0.297673,0.315381,0.337092,0.358024,0.379078,0.401502,0.426114,0.453869,0.480832


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0169431,-0.0296041,-0.0552324,-0.0751632,-0.0927613,-0.107445,-0.125241,-0.141486,-0.159031,-0.176534,-0.198007,-0.218784,-0.238844,-0.261815,-0.289569
0.25,0.0431181,0.027225,0.0205409,0.016728,0.0140392,0.0122988,0.0106893,0.00933692,0.00816155,0.00678852,0.00593244,0.00471739,0.00402623,0.00366512,0.00276304
0.5,0.0553983,0.0527796,0.0526876,0.0536461,0.0544795,0.0559131,0.0573078,0.0580625,0.0591961,0.0600989,0.0610958,0.0614094,0.0617675,0.0619747,0.0624636
0.75,0.0676087,0.0784682,0.0842934,0.0892268,0.0933744,0.0975418,0.101199,0.104466,0.107332,0.109909,0.112001,0.11388,0.115558,0.117004,0.118234
0.98,0.0938025,0.133012,0.154155,0.170025,0.184966,0.199422,0.212648,0.225517,0.240666,0.254051,0.268321,0.282411,0.296919,0.310395,0.326066


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0483945,-0.0860746,-0.103907,-0.119261,-0.132505,-0.147431,-0.162471,-0.178141,-0.19363,-0.210244,-0.230641,-0.248479,-0.273379,-0.298886,-0.324784
0.25,-0.022265,-0.0307146,-0.0333339,-0.0352938,-0.0369923,-0.0383627,-0.0399039,-0.0412239,-0.0421842,-0.0433653,-0.044737,-0.0453221,-0.0464378,-0.0472492,-0.0476146
0.5,-0.0103952,-0.00604693,-0.00342974,-0.00126428,0.00029341,0.00176225,0.00275933,0.00369492,0.00471608,0.00539749,0.0053007,0.00580305,0.00566863,0.00621473,0.00643744
0.75,0.00151699,0.0181393,0.0259932,0.0318508,0.0361206,0.0403452,0.0433563,0.0463961,0.0489941,0.0507677,0.0523046,0.0537263,0.0548813,0.0559709,0.0568238
0.98,0.0274656,0.0714333,0.0914031,0.106823,0.121626,0.133973,0.145327,0.157984,0.169981,0.182114,0.193375,0.207529,0.220222,0.233554,0.246642


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.00548905,-0.0161981,-0.0287376,-0.0360077,-0.0433901,-0.050242,-0.0565864,-0.0614255,-0.0670889,-0.0731098,-0.0799028,-0.0864772,-0.0920606,-0.100167,-0.107377
0.25,0.022155,0.0179375,0.0152763,0.0133783,0.0118387,0.0102457,0.0092307,0.00810685,0.00708394,0.0062825,0.00577478,0.00505849,0.00453295,0.00379966,0.00357804
0.5,0.0299539,0.0318906,0.0316654,0.0312575,0.0309657,0.030587,0.0304278,0.0298334,0.0297301,0.0294964,0.0292076,0.0292861,0.0291001,0.0288857,0.0289916
0.75,0.037969,0.0467257,0.0492193,0.0506588,0.0515978,0.0524719,0.053387,0.0538624,0.0545651,0.0549887,0.0551971,0.0558203,0.0560431,0.0565137,0.0564484
0.98,0.0562529,0.0869353,0.100439,0.110142,0.117938,0.12492,0.132499,0.140359,0.14968,0.15726,0.16683,0.175858,0.187007,0.196501,0.208721


In [27]:
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[4,20,40,60]] ./ [1 5 10 15]),[.05, .25, .5, .75, .95], string(names[n]))    
end

,1,2,3,4
0.05,-0.0847338,-0.00314259,0.013434,0.0239475
0.25,0.0374589,0.0437379,0.0440843,0.046092
0.5,0.137984,0.0787286,0.0652921,0.0621107
0.75,0.246016,0.114012,0.0868185,0.0787572
0.95,0.420067,0.166814,0.119904,0.103288


,1,2,3,4
0.05,-0.140182,-0.0320725,-0.0129672,-0.00939393
0.25,-0.0722432,-0.0107642,-0.000932799,0.00066611
0.5,-0.0193813,0.00432851,0.00776081,0.00739053
0.75,0.035035,0.0195995,0.0163063,0.0142811
0.95,0.117523,0.0421216,0.029007,0.0244454


,1,2,3,4
0.05,-0.0123747,-0.00696491,-0.0126797,-0.0156149
0.25,0.00227694,0.00302321,-0.00139625,-0.00420326
0.5,0.0123856,0.0102394,0.00626262,0.00359814
0.75,0.0225914,0.0174226,0.0142489,0.0115957
0.95,0.0374292,0.0276361,0.0255367,0.0231666


In [28]:
returns_exp = expm1.(scenarios[[1,2,3],:,:])
for n in 1:3
    s_ann = annualise(returns_exp[n,:,2:end],4)
    print_percentiles(s_ann,[.1, .3, .5, .7, .9], string(names[n]))
end


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.0352533,-0.104495,-0.1186,-0.12465,-0.12648,-0.133901,-0.13052,-0.13363,-0.132143,-0.130043,-0.129023,-0.13206,-0.124387,-0.12514,-0.127952
0.3,0.0660326,0.00451676,-0.00770669,-0.0149831,-0.0184142,-0.0224522,-0.0173066,-0.0220483,-0.0176186,-0.0193931,-0.016906,-0.0192532,-0.0130999,-0.013884,-0.0167445
0.5,0.140321,0.0850213,0.0707888,0.0632035,0.0607843,0.0561174,0.0574514,0.061562,0.0591275,0.0625848,0.0615783,0.0601529,0.0657975,0.063789,0.0641936
0.7,0.213734,0.163148,0.15268,0.145062,0.141766,0.138237,0.13978,0.142224,0.143131,0.14216,0.143183,0.139869,0.146515,0.143797,0.145651
0.9,0.323097,0.28303,0.269149,0.265289,0.264045,0.25619,0.261788,0.263562,0.262205,0.26134,0.262444,0.259314,0.272023,0.259801,0.269979


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.118969,-0.0987452,-0.098623,-0.095371,-0.100097,-0.10155,-0.101685,-0.102711,-0.102368,-0.102188,-0.106257,-0.105449,-0.106177,-0.103811,-0.107942
0.3,-0.0600043,-0.0346651,-0.0347714,-0.0313706,-0.0312449,-0.0307125,-0.0319449,-0.0349864,-0.0339228,-0.0351663,-0.0368564,-0.0386801,-0.0380994,-0.0372618,-0.0396797
0.5,-0.0164677,0.00783337,0.0127657,0.0149293,0.0157598,0.016375,0.0153845,0.0132117,0.0142729,0.0133129,0.0122578,0.00872329,0.00986652,0.0114393,0.0107501
0.7,0.0249208,0.0544473,0.060418,0.0631172,0.0630344,0.0626673,0.0635324,0.0609218,0.0621538,0.0624485,0.0597831,0.057645,0.057051,0.058727,0.0585174
0.9,0.0874492,0.122427,0.128664,0.133055,0.132645,0.134912,0.135507,0.131285,0.131629,0.129734,0.129752,0.128904,0.127647,0.128664,0.127449


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.00679367,-0.0089289,-0.014072,-0.0180343,-0.0208072,-0.0236481,-0.0246181,-0.0277991,-0.0287676,-0.0297486,-0.0307782,-0.0314252,-0.031828,-0.0317702,-0.0325941
0.3,0.00460164,0.00422588,0.000550083,-0.00211618,-0.00454503,-0.00650628,-0.00792315,-0.0103779,-0.0108328,-0.0120999,-0.0132746,-0.0135544,-0.0137448,-0.0144457,-0.0150236
0.5,0.012409,0.0135313,0.0105157,0.00863792,0.0066159,0.00531996,0.003761,0.00217196,0.00127043,0.000603189,-0.000878405,-0.00148052,-0.0015175,-0.00177712,-0.00276876
0.7,0.0201808,0.0222591,0.0206281,0.0189552,0.0178356,0.0166229,0.0154358,0.0139557,0.0137607,0.0127126,0.011888,0.0111469,0.0106816,0.0107931,0.00996129
0.9,0.0313664,0.0354498,0.0351999,0.0347515,0.0342389,0.0335391,0.0322909,0.0316492,0.0311678,0.0308853,0.0297261,0.0298063,0.0289202,0.0285023,0.0281166


In [41]:
kurtosis(rand(Normal(0,1),1000_000))

0.0037992201690122407